# Digital Twin Fairways
WIP to use the OpenTNSim engine

In [1]:
import datetime
import json
import logging
import time

import dtv_backend.berthing

# the simpy processes and objects
import dtv_backend.compat

# library to load the fairway information network
import dtv_backend.fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate
import geojson
import networkx as nx
import pandas as pd
import geopandas as gpd
import shapely
import simpy
from networkx.readwrite import json_graph

# opentnsim import
from opentnsim import core

import plotly.express as px


# reload for debugging purposes
%load_ext autoreload
%autoreload 2

logger = logging.getLogger("notebook")

### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open("../../dtv_backend/tests/user/2022-09-15-config.json") as f:
    config = geojson.load(f)

### Simulate with ``v3_run``

In [3]:
def v3_dtv_backend_simulate(config):
    result = dtv_backend.simulate.v3_run(config)
    env = result['env']
    logbook = result['operator'].logbook
    log_df = pd.DataFrame(logbook)
    log_json = dtv_backend.postprocessing.log2json(log_df)
    
    response = {
        "log": log_json,
        "config": config,
        "env": {
            "epoch": env.epoch.timestamp(),
            "epoch_iso": env.epoch.isoformat(),
            "now": env.epoch,
            "now_iso": datetime.datetime.fromtimestamp(env.now).isoformat(),
        },
    }
    return response
    

In [4]:
config['sites'][1]['properties']

{'Wkt': 'POINT (4.4389420728918099 51.9065380788739006)',
 'X': 4.43894207289181,
 'Y': 51.9065380788739,
 'n': '8865146',
 'name': '8865146',
 'cargo_type': 'Dry Bulk',
 'capacity': 10000,
 'level': 0,
 'loading_rate': 198,
 'loading_rate_variation': 102}

In [5]:
resp = v3_dtv_backend_simulate(config)

/Users/baart_f/src/digitaltwin-waterway/dtv_backend/dtv_backend/fis.py:118: ShapelyDeprecationWarning:

The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.



{"geometry": {"coordinates": [[4.091169, 51.975139], [4.090019, 51.972115]], "type": "LineString"}, "id": 0, "properties": {"Beam 10% percentile [m]": 8.18, "Beam 50% percentile [m]": 8.2, "Beam 90% percentile [m]": 8.24, "Beam [m]": 8.2, "CEMT-class": "III", "Description (Dutch)": "Verlengde Dortmund (L > 74m)", "Description (English)": "Verlengde Dortmunder", "Draught average [m]": 2.4, "Draught empty [m]": 1.2, "Draught loaded [m]": 2.7, "Height average [m]": 4.95, "Length 10% percentile [m]": 76.95, "Length 50% percentile [m]": 79.97, "Length 90% percentile [m]": 84.96, "Length [m]": 85, "Load Weight average [ton]": 995, "Load weight maximum [ton]": 1171, "RWS-class": "M5", "Vessel type": "Motorvessel", "capacity": 1000, "count": 1, "image": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQRvKRniAxUXUWzmByw7CRFYD5fTqOtFTDVkw&usqp=CAU", "name": "Verlengde Dortmunder", "speed": 3}, "type": "Feature"}


KeyError: 'Velocity [m/s]'

In [ ]:
gdf = gpd.GeoDataFrame.from_features(resp['log'])
gdf.head(n=20)

In [ ]:
fig = px.timeline(
    gdf, x_start="Start", x_end="Stop", y="Name", color="Actor", opacity=0.3
)

fig.update_yaxes(autorange="reversed")
